<H2>Part 1: Create basic dataframe with Neighbourhoods</H2>

<b>Import all libraries:</b>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

<b>Scrape webpage using Beautiful Soup:</b>

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
html = pd.read_html(str(table))
json = html[0].to_json(orient='records')
neighbors = pd.read_json(json)
neighbors = neighbors [["Postcode", "Borough", "Neighbourhood"]]
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b>Clean not assigned Boroughs:</b>

In [3]:
neighbors.drop(neighbors.loc[neighbors['Borough']=="Not assigned"].index, inplace=True)
neighbors = neighbors.reset_index(drop=True)
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<b>Group by Postcode separating Neighbourhoods by comma:</b>

In [4]:
neighbors = neighbors.groupby('Postcode').agg({'Borough' : 'first', 'Neighbourhood' : ', '.join}).reset_index().reindex(columns=neighbors.columns)
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Rows with "Not assigned" Neighbourhoods will have the value of the Borough</b>

In [5]:
neighbors["Neighbourhood"].replace("Not assigned",neighbors["Borough"],inplace=True)
neighbors.loc[neighbors['Postcode']=='M9A'] #Check the replaced row

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


<b>Check the final dataframe:</b>

In [6]:
neighbors

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<b>Check the shape of the dataframe:</b>

In [7]:
neighbors.shape

(103, 3)

<H2>Part 2: Get latitud and longitud of each neighbourhood</H2>

<b>Install and Import libraries:</b>

<i>Note: I did not use the geocoder python package, it would not work for me. I used an alternate library called "pgeocode". Final values could differ slightly from other works done using the original library.</i>

In [8]:
!pip install pgeocode
import pgeocode

<b>Define a function to get coordinates:</b>

In [9]:
def get_coordinates(pcode):
    geodata = pgeocode.Nominatim('ca')
    data = geodata.query_postal_code(pcode)
    lat = data.latitude
    long = data.longitude
    return lat,long

<b>Get coordinates for each row of the dataframe:</b>

In [10]:
neighbors['Latitude'], neighbors['Longitude'] = 0.0,0.0

for i in range(0,len(neighbors)):
    neighbors['Latitude'].loc[i],neighbors['Longitude'].loc[i] = get_coordinates(neighbors.iloc[i][0])

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<b>Drop values not found:</b>

In [11]:
neighbors.dropna(subset=['Latitude', 'Longitude'], inplace = True)
neighbors

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7298,-79.2639
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6952,-79.2646


<H2>Part 3: Exploring and Clustering Neighborhooods</H2>

<b>Import libraries:</b>

In [12]:
!pip install folium

import folium # map rendering library

In [13]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<b>Get only boroughs with the word Toronto:</b>

In [14]:
neighbors = neighbors[neighbors['Borough'].str.contains("Toronto")]
neighbors = neighbors.reset_index(drop=True)
neighbors

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935
5,M4P,Central Toronto,Davisville North,43.7135,-79.3887
6,M4R,Central Toronto,North Toronto West,43.7143,-79.4065
7,M4S,Central Toronto,Davisville,43.7020,-79.3853
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6861,-79.4025


<b>Create a map of the area:</b>

In [15]:
map_toronto = folium.Map(location=["43.707454", "-79.340665"], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbors['Latitude'], neighbors['Longitude'], neighbors['Borough'], neighbors['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b>Define Foursquare Credentials(hidden):</b>

In [39]:
# The code was removed by Watson Studio for sharing.

<b>Define function to get nearby Venues for each neighborhood:</b>

In [17]:
radius = 500
limit = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Get Dataframe with all venue data:</b>

In [18]:
toronto_venues = getNearbyVenues(names=neighbors['Neighbourhood'],
                                   latitudes=neighbors['Latitude'],
                                   longitudes=neighbors['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

<b>Check Shape of the Dataframe:</b>

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(1634, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub


<b>Count Venues per Neighbourhood:</b>

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,91,91,91,91,91,91
"Brockton, Exhibition Place, Parkdale Village",38,38,38,38,38,38
Business Reply Mail Processing Centre 969 Eastern,13,13,13,13,13,13
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",58,58,58,58,58,58
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,97,97,97,97,97,97
"Chinatown, Grange Park, Kensington Market",81,81,81,81,81,81
Christie,12,12,12,12,12,12


<b>Check unique categories:</b>

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


<b>Analyze each neighborhood:</b>

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_onehot.shape

(1634, 214)

<b>Group rows by neighborhood and take the mean of the frequency of occurrence of each category:</b>

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.010000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.000000,...,0.000000,0.00,0.00,0.010989,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.026316,0.000000,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,Central Bay Street,0.000000,0.000000,0.000000,0.010309,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.010309,0.010309,0.00,0.010309,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.037037,0.012346,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.049383,0.000000,0.049383,0.00,0.012346,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
9,Church and Wellesley,0.011494,0.000000,0.011494,0.011494,0.000000,0.011494,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.011494,0.00,0.000000,0.011494,0.00


In [25]:
toronto_grouped.shape

(39, 214)

<b>Print each neighborhood along with the top 5 most common venues:</b>

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.06
2                Hotel  0.04
3           Steakhouse  0.04
4  Japanese Restaurant  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2                Hotel  0.05
3               Bakery  0.04
4  Japanese Restaurant  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.05
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                 venue  freq
0          Coffee Shop  0.15
1       Breakfast Spot  0.08
2            Bookstore  0.08
3           Restaurant  0.08
4  Japanese Restaurant  0.08


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----

<b>Write a function to sort the venues in descending order:</b>

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Create a new dataframe and display the top 5 venues for each neighborhood:</b>

In [28]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Steakhouse,Bar
1,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Cocktail Bar
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Thrift / Vintage Store,Bar,Café,Breakfast Spot
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Yoga Studio,Department Store,Italian Restaurant,Japanese Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Café,Bar,Park
5,"Cabbagetown, St. James Town",Café,Coffee Shop,Pizza Place,Park,Italian Restaurant
6,Central Bay Street,Coffee Shop,Bakery,Japanese Restaurant,Café,Clothing Store
7,"Chinatown, Grange Park, Kensington Market",Café,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant
8,Christie,Grocery Store,Café,Park,Bank,Candy Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Burger Joint,Sushi Restaurant


<b>Cluster neighborhoods using k-means:</b>

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 0, 4], dtype=int32)

<b>Create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood</b>

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [31]:
toronto_merged = neighbors

# merge neighborhoods_venues_sorted with neighbors to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Cheese Shop,Park,Health Food Store,Gastropub
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Coffee Shop,Yoga Studio
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6693,-79.3155,4,Park,Ice Cream Shop,Pet Store,Movie Theater,Pizza Place
3,M4M,East Toronto,Studio District,43.6561,-79.3406,4,Park,Coffee Shop,Performing Arts Venue,Diner,Garden Center
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,3,Photography Studio,Park,Women's Store,Discount Store,Farmers Market


In [32]:
#toronto_merged['Cluster Labels'] = pd.to_numeric(toronto_merged['Cluster Labels'], errors='coerce')
#toronto_merged

#toronto_merged.dropna(subset=['Cluster Labels'], inplace = True)
#toronto_merged

<b>Visualize the resulting clusters:</b>

In [33]:
# create map
map_toronto = folium.Map(location=["43.6519", "-79.3874"], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

<b>See each cluster:</b>

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,The Beaches,0,Pub,Cheese Shop,Park,Health Food Store,Gastropub
5,Davisville North,0,Dog Run,Park,Gym,Breakfast Spot,Food & Drink Shop
6,North Toronto West,0,Gym Pool,Park,Playground,Garden,Diner
8,"Moore Park, Summerhill East",0,Park,Grocery Store,Playground,Thai Restaurant,Gym
10,Rosedale,0,Park,Grocery Store,Playground,Candy Store,Discount Store
19,"Harbourfront East, Toronto Islands, Union Station",0,Harbor / Marina,Café,Music Venue,Park,Grocery Store
23,"Forest Hill North, Forest Hill West",0,Park,Business Service,Trail,Furniture / Home Store,Dog Run
30,Christie,0,Grocery Store,Café,Park,Bank,Candy Store
31,"Dovercourt Village, Dufferin",0,Park,Bakery,Gym / Fitness Center,Pharmacy,Middle Eastern Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Roselawn,1,IT Services,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,"High Park, The Junction South",2,Park,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Lawrence Park,3,Photography Studio,Park,Women's Store,Discount Store,Farmers Market


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"The Danforth West, Riverdale",4,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Coffee Shop,Yoga Studio
2,"The Beaches West, India Bazaar",4,Park,Ice Cream Shop,Pet Store,Movie Theater,Pizza Place
3,Studio District,4,Park,Coffee Shop,Performing Arts Venue,Diner,Garden Center
7,Davisville,4,Sandwich Place,American Restaurant,Café,Italian Restaurant,Coffee Shop
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",4,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Cupcake Shop
11,"Cabbagetown, St. James Town",4,Café,Coffee Shop,Pizza Place,Park,Italian Restaurant
12,Church and Wellesley,4,Coffee Shop,Japanese Restaurant,Gay Bar,Burger Joint,Sushi Restaurant
13,Harbourfront,4,Coffee Shop,Breakfast Spot,Yoga Studio,Beer Store,Health Food Store
14,"Ryerson, Garden District",4,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant
15,St. James Town,4,Coffee Shop,Café,Restaurant,Bakery,Seafood Restaurant


<H2><i>The insight that can be retrieved from observing the clustering, is that there are two main groups of neighborhoods in Toronto. The ones classified within Cluster 0 (with venues mostly related to Parks, Gyms and Playgrounds) and Cluster 4 (with venues mostly related to Coffee Shops and Restaurants).</H2></i>